# Library

In [1]:
import os
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import random
import shutil
import pickle

from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# #Function

In [2]:
Index2Label = {0: 'Background',
               1: 'Head',
               2: 'Torso',
               3: 'Upper_Arms',
               4: 'Lower_Arms',
               5: 'Upper_Legs',
               6: 'Lower_Legs'}

Label2Index = {'Background': 0,
               'Head': 1,
               'Torso': 2,
               'Upper_Arms': 3,
               'Lower_Arms': 4,
               'Upper_Legs': 5,
               'Lower_Legs': 6}

# Label2Index = {v: k for k, v in Index2Label.items()}

In [3]:
def create_images_path(folder_path):
    folder_path = folder_path
    imges = os.listdir(folder_path)
    images_path = [os.path.join(folder_path, img) for img in imges]
    return images_path


def visualize_images(images_path, visualize=1, rotate=0): # Outputs\\01_01_F010_02.png, rotate 회전각 입력
    # Output
    images_names = []
    images_array = []

    for img in tqdm(images_path, desc="Processing images"):
        image_path = img
        image_name = os.path.basename(image_path)

        image = Image.open(image_path)

        # 이미지 회전 메타데이터를 자동으로 해석하여 회전시킴
        image = image.rotate(rotate, expand=True)  # 0은 회전 각도, expand=True는 이미지 크기 조정 (0 / 270)

        images_names.append(image_name)
        image_array = np.array(image)
        images_array.append(image_array)

        # Visualize Images -------------------> Option(On/Off)
        if visualize:

            print("Size of Image:", image.size)
            print("Mode of Image:", image.mode) # (RGB/L/P)

            # print("Shape of Image (Numpy ):", )


            plt.imshow(image)
            plt.title(f'{image_name}')
            plt.axis('off')  # 축 제거
            plt.show()
    else:
        pass

    return images_names, images_array


def palette_to_one_hot(palette_image_path, num_classes=7, imge_info=True, visualize=True):
    """
    팔레트 이미지를 원핫인코딩하여 (2940, 1960, 7) 형태의 이미지로 변환하는 함수

    Args:
    palette_image_path (str): 세그멘테이션된 P 모드의 PIL 팔레트 이미지 경로

    Returns:
    np.ndarray: 원핫인코딩된 이미지
    """
    # 클래스 수 정의 (배경 포함 1개)
    num_classes = num_classes

    # PIL 2 Numpy
    palette_image = Image.open(palette_image_path)
    palette_image_array = np.array(palette_image)

    # 원핫인코딩된 이미지를 저장할 배열 생성
    one_hot_encoded_image = np.zeros((*palette_image_array.shape, num_classes), dtype=np.uint8)

    # 각 픽셀을 원핫인코딩된 형태로 변환
    for i in range(palette_image_array.shape[0]): # 2940
        for j in range(palette_image_array.shape[1]): # 1960
            pixel_value = palette_image_array[i, j]  # 현재 픽셀의 값
            one_hot_encoded_image[i, j, pixel_value] = 1  # 해당 클래스에 대응하는 채널에 1 할당

    if imge_info:
        # 이미지 이름 출력
        image_name = os.path.basename(palette_image_path)
        print("Image Name:", image_name)
        print("Original Image Shape:", palette_image_array.shape)
        print("Transformed Image Shape:", one_hot_encoded_image.shape)
        print()

    if visualize:
        visualize_one_hot_encoded_image(one_hot_encoded_image,  Index2Label=Index2Label)

    return one_hot_encoded_image


def visualize_one_hot_encoded_image(one_hot_encoded_image, Index2Label=Index2Label):
    """
    원핫인코딩된 이미지를 시각화하고 클래스별로 따로 시각화하여 보여주는 함수

    Args:
    one_hot_encoded_image (np.ndarray): 원핫인코딩된 이미지

    Returns:
    None
    """
    # 이미지의 형태 출력
    print("Transformed Image Shape:", one_hot_encoded_image.shape)

    # 전체 이미지 시각화
    plt.figure(figsize=(8, 8))
    plt.imshow(one_hot_encoded_image.argmax(axis=2), cmap='viridis', interpolation='nearest')
    plt.title('Original Palette Image')
    plt.axis('off')
    plt.show()

    # 클래스별로 따로 시각화
    num_classes = one_hot_encoded_image.shape[2]
    for class_index in range(num_classes):
        class_image = one_hot_encoded_image[:, :, class_index]
        plt.figure(figsize=(8, 8))
        plt.imshow(class_image, cmap='gray')
        plt.title(f'Class: {class_index} {Index2Label[class_index]}')
        plt.axis('off')
        plt.show()


def save_images_pkl_with_one_hot_encoding(images_paths, output_folder_path, num_classes=7, imge_info=True, visualize=False):
    """
    이미지를 원핫인코딩하여 새로운 폴더에 저장하는 함수

    Args:
    images_paths (list): 이미지 파일 경로의 리스트
    output_folder_path (str): 이미지를 저장할 폴더 이름(현재 경로에 저장)
    num_classes (int): segmentation 세그멘테이션 클래스 개수(배경 포함)
    imge_info (bool): 이미지 정보 출력 토글(default: True)
    visualize (bool): 이미지 시각화 토글(default: False)

    Returns:
    None
    """
    # 새로운 폴더 생성
    os.makedirs(output_folder_path, exist_ok=True)

    # one_hot_encoded_images
    one_hot_encoded_images = []

    # 원핫 인코딩 수행
    for img_path in tqdm(images_paths, desc='Encoding images'):

        # PIL Pallete 2 Numpy
        one_hot_encoded_image = palette_to_one_hot(img_path, num_classes=num_classes, imge_info=imge_info, visualize=visualize)
        one_hot_encoded_images.append(one_hot_encoded_image)

    # 저장할 경로 설정
    save_path = os.path.join(output_folder_path, f"one_hot_encoded_images_array.pkl")

    # 이미지 데이터를 피클로 저장
    with open(save_path, 'wb') as f:
        pickle.dump(one_hot_encoded_images, f)

# #Load Data

In [4]:
!pip install gdown

피클

In [ ]:
# 01.pkl
# !gdown --id 1-JCaKXiWE37MoyaL2r3mxpO1Tzecev5M
# !gdown --id 1TO3kT3FsBJCZ6NMDaxGPsrAlAbBhxC10
# !gdown --id 1TO3kT3FsBJCZ6NMDaxGPsrAlAbBhxC10
# !gdown --id 1TO3kT3FsBJCZ6NMDaxGPsrAlAbBhxC10
# !gdown --id 1TO3kT3FsBJCZ6NMDaxGPsrAlAbBhxC10
# !gdown 130nPl5Sun1CftENwIY0w0S6RuvpKHbNV

Failed to retrieve file url:

	Too many users have viewed or downloaded this file recently. Please
	try accessing the file again later. If the file you are trying to
	access is particularly large or is shared with many people, it may
	take up to 24 hours to be able to view or download the file. If you
	still can't access a file after 24 hours, contact your domain
	administrator.

You may still be able to access the file from the browser:

	https://drive.google.com/uc?id=130nPl5Sun1CftENwIY0w0S6RuvpKHbNV

but Gdown can't. Please check connections and permissions.


이미지

In [11]:
# !gdown --id 1wFfIf6aOTvOtb6uR6wxKfifUHb2LNuhq

/usr/local/lib/python3.10/dist-packages/gdown/cli.py:138: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Failed to retrieve file url:

	Too many users have viewed or downloaded this file recently. Please
	try accessing the file again later. If the file you are trying to
	access is particularly large or is shared with many people, it may
	take up to 24 hours to be able to view or download the file. If you
	still can't access a file after 24 hours, contact your domain
	administrator.

You may still be able to access the file from the browser:

	https://drive.google.com/uc?id=1wFfIf6aOTvOtb6uR6wxKfifUHb2LNuhq

but Gdown can't. Please check connections and permissions.


In [12]:
# !unzip img.zip

unzip:  cannot find or open img.zip, img.zip.zip or img.zip.ZIP.


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
pkl_path = "/content/drive/MyDrive/one_hot_encoded_images_array_01.pkl"

with open(pkl_path, 'rb') as f:
  seg_imges = pickle.load(f)

seg_array = np.array(seg_imges)

print(type(seg_imges), len(seg_imges))
print(type(seg_array), seg_array.shape)

<class 'list'> 872
<class 'numpy.ndarray'> (872, 2940, 1960, 7)


In [7]:
y = []

for seg_imge in seg_array:
    resized_array = resized_padded_image = tf.image.resize_with_pad(seg_imge, target_height=512, target_width=512).numpy()
    y.append(resized_array)

y = np.array(y)
y.shape

(872, 512, 512, 7)

In [8]:
# 배열을 삭제하여 메모리 확보
del seg_imges
del seg_array

In [ ]:
# import zipfile

# # 압축 파일 경로
# zip_file_path = "/content/drive/MyDrive/img.zip"

# # 압축 해제할 폴더 경로
# extract_folder_path = "/content/drive/MyDrive/img"

# # 압축 파일을 열고 압축 해제
# with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
#     zip_ref.extractall(extract_folder_path)

In [9]:
!gdown --id 1ZzrjGvnLrQbQX7_XCC9La7sPHwLsmji7

/usr/local/lib/python3.10/dist-packages/gdown/cli.py:138: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1ZzrjGvnLrQbQX7_XCC9La7sPHwLsmji7
From (redirected): https://drive.google.com/uc?id=1ZzrjGvnLrQbQX7_XCC9La7sPHwLsmji7&confirm=t&uuid=835f5251-9a50-4007-acde-cd0df94699ce
To: /content/origin.zip
100% 7.92G/7.92G [00:50<00:00, 156MB/s]


In [10]:
!unzip /content/origin.zip -d /content/Origin_image

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
  inflating: /content/Origin_image/img/24/04_05_F448_24.jpg  
  inflating: /content/Origin_image/img/24/04_05_M002_24.jpg  
  inflating: /content/Origin_image/img/24/04_05_M004_24.jpg  
  inflating: /content/Origin_image/img/24/04_06_F001_24.jpg  
  inflating: /content/Origin_image/img/24/04_06_F003_24.jpg  
  inflating: /content/Origin_image/img/24/04_06_F004_24.jpg  
  inflating: /content/Origin_image/img/24/04_06_F020_24.jpg  
  inflating: /content/Origin_image/img/24/04_06_F023_24.jpg  
  inflating: /content/Origin_image/img/24/04_06_F025_24.jpg  
  inflating: /content/Origin_image/img/24/04_06_F395_24.jpg  
  inflating: /content/Origin_image/img/24/04_06_F396_24.jpg  
  inflating: /content/Origin_image/img/24/04_06_F401_24.jpg  
  inflating: /content/Origin_image/img/24/04_06_F438_24.jpg  
  inflating: /content/Origin_image/img/24/04_06_F439_24.jpg  
  inflating: /content/Origin_image/img/24/04_06_F444_24.jpg  
  inflating: /content/Origin_image

In [11]:
origin_images_path = create_images_path("/content/Origin_image/img/01")
len(origin_images_path)

872

In [12]:
origin_images_path.sort()

In [13]:
origin_path = origin_images_path[:100]
crop_y = y[:100]

del y

In [18]:
imges_name, imges_array = visualize_images(origin_path, visualize=False, rotate=270)

Processing images: 100%|██████████| 100/100 [00:07<00:00, 13.34it/s]


In [19]:
origin_array = np.array(imges_array)

In [20]:
del imges_array

In [21]:
origin_array.shape

(100, 2940, 1960, 3)

In [24]:
X = []

for image_array in origin_array:
    resized_array = resized_padded_image = tf.image.resize_with_pad(image_array, target_height=512, target_width=512).numpy()
    X.append(resized_array)

X = np.array(X)
X.shape

(100, 512, 512, 3)

In [25]:
crop_y.shape

(100, 512, 512, 7)

In [26]:
y = crop_y

X.shape, y.shape

((100, 512, 512, 3), (100, 512, 512, 7))

# #Model
- Standard UNET

In [27]:
def conv_block(inputs, filters, kernel_size=3):
    x = layers.Conv2D(filters, kernel_size, activation='relu', padding='same')(inputs)
    x = layers.Conv2D(filters, kernel_size, activation='relu', padding='same')(x)
    return x

def upconv_block(inputs, filters, kernel_size=2):
    x = layers.UpSampling2D(size=(2, 2))(inputs)
    x = layers.Conv2D(filters, kernel_size, activation='relu', padding='same')(x)
    return x

def unet(img_size, num_classes):
    inputs = tf.keras.Input(shape=img_size + (3,))

    # Contracting Path
    c1 = conv_block(inputs, 64, kernel_size=3)
    p1 = layers.MaxPooling2D(pool_size=(2, 2))(c1)

    c2 = conv_block(p1, 128, kernel_size=3)
    p2 = layers.MaxPooling2D(pool_size=(2, 2))(c2)

    c3 = conv_block(p2, 256, kernel_size=3)
    p3 = layers.MaxPooling2D(pool_size=(2, 2))(c3)

    c4 = conv_block(p3, 512, kernel_size=3)
    p4 = layers.MaxPooling2D(pool_size=(2, 2))(c4)

    # Bottom
    b = conv_block(p4, 1024, kernel_size=3)

    # Expanding Path
    u1 = upconv_block(b, 512, kernel_size=2)
    u1_concat = layers.Concatenate()([u1, c4])
    c5 = conv_block(u1_concat, 512, kernel_size=3)

    u2 = upconv_block(c5, 256, kernel_size=2)
    u2_concat = layers.Concatenate()([u2, c3])
    c6 = conv_block(u2_concat, 256, kernel_size=3)

    u3 = upconv_block(c6, 128, kernel_size=2)
    u3_concat = layers.Concatenate()([u3, c2])
    c7 = conv_block(u3_concat, 128, kernel_size=3)

    u4 = upconv_block(c7, 64, kernel_size=2)
    u4_concat = layers.Concatenate()([u4, c1])
    c8 = conv_block(u4_concat, 64, kernel_size=3)

    outputs = layers.Conv2D(num_classes, 1, activation='softmax')(c8)

    return tf.keras.Model(inputs, outputs)

In [ ]:
# Clearing previous sessions
# tf.keras.backend.clear_session()

In [29]:
img_size = (512, 512)
num_classes = 7

In [30]:
unet = unet(img_size, num_classes)

unet.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
unet.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 512, 512, 3)]        0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 512, 512, 64)         1792      ['input_1[0][0]']             
                                                                                                  
 conv2d_1 (Conv2D)           (None, 512, 512, 64)         36928     ['conv2d[0][0]']              
                                                                                                  
 max_pooling2d (MaxPooling2  (None, 256, 256, 64)         0         ['conv2d_1[0][0]']            
 D)                                                                                           

# Training(02.29.Thu)

In [31]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((80, 512, 512, 3), (20, 512, 512, 3), (80, 512, 512, 7), (20, 512, 512, 7))

In [33]:
print('Number of train images', X_train.shape[0])

num_X_train = X_train.shape[0]
batch_size = 20 # batch_size 일반적으로, 배치 사이즈는 2의 n 거듭제곱으로 잡음
num_batch = X_train.shape[0] // batch_size

print('num_X_train:', num_X_train)
print('batch_size:', batch_size)
print('num_batch:', num_batch)
print(f'residual = {num_X_train}-{batch_size}x{num_batch} =', num_X_train-batch_size*num_batch)

Number of train images 80
num_X_train: 80
batch_size: 20
num_batch: 4
residual = 80-20x4 = 0


In [34]:
# Call Backs: Early Stopping
early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

In [35]:
# Model fitting
hist = unet.fit(X_train, y_train, epochs = 5, validation_split=0.2 , batch_size=batch_size, callbacks=[early], verbose=1)


Epoch 1/5
4/4 [==============================] - 79s 7s/step - loss: 24.1985 - accuracy: 0.7536 - val_loss: 7.0959 - val_accuracy: 0.0139
Epoch 2/5
4/4 [==============================] - 2s 499ms/step - loss: 2.5657 - accuracy: 0.4253 - val_loss: 0.5169 - val_accuracy: 0.6149
Epoch 3/5
4/4 [==============================] - 2s 500ms/step - loss: 0.4738 - accuracy: 0.5781 - val_loss: 0.4089 - val_accuracy: 0.6096
Epoch 4/5
4/4 [==============================] - 2s 499ms/step - loss: 1.8098 - accuracy: 0.4432 - val_loss: 0.8288 - val_accuracy: 0.6167
Epoch 5/5
4/4 [==============================] - 2s 498ms/step - loss: 1.1236 - accuracy: 0.5815 - val_loss: 0.9698 - val_accuracy: 0.6164


In [36]:
# Evaluate
unet.evaluate(X_test, y_test)

1/1 [==============================] - 0s 282ms/step - loss: 1.0368 - accuracy: 0.6169


[1.0367671251296997, 0.6169496774673462]

# Train, Validation Grapgh

In [37]:
import plotly.graph_objects as go
import plotly.express as px

def plot_hist(hist):
    fig = go.Figure()

    # Train and Validation Accuracy
    fig.add_trace(go.Scatter(x=[epoch + 1 for epoch in hist.epoch], y=hist.history['accuracy'], mode='lines+markers', name='Train Accuracy', line=dict(color='rgb(190, 82, 243)')))
    fig.add_trace(go.Scatter(x=[epoch + 1 for epoch in hist.epoch], y=hist.history['val_accuracy'], mode='lines+markers', name='Validation Accuracy', line=dict(color='rgb(128, 177, 211)')))

    # Train and Validation Loss with dashed lines
    fig.add_trace(go.Scatter(x=[epoch + 1 for epoch in hist.epoch], y=hist.history['loss'], mode='lines+markers', name='Train Loss', line=dict(color='rgb(244, 109, 67)', dash='dash')))
    fig.add_trace(go.Scatter(x=[epoch + 1 for epoch in hist.epoch], y=hist.history['val_loss'], mode='lines+markers', name='Validation Loss', line=dict(color='rgb(103, 203, 203)', dash='dash')))

    # Update layout
    fig.update_layout(title='Training History',
                      xaxis=dict(title='Epoch'),
                      yaxis=dict(title='Value'),
                      template='plotly_dark')  # plotly_dark / plotly_white

    # Show plot
    fig.show()

In [38]:
# Assuming you have a 'hist' object after training
plot_hist(hist)